In [6]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.inspection import permutation_importance
import statsmodels.api as sm

def clean_data(df, drop_exp_above_1=True):
    if drop_exp_above_1:
        df = df[df["Exposure"] <= 1]
    df = df.dropna()
    return df

def feature_selection(df):
    df = df.drop(columns=["IDpol", "Density"])
    
    return df
def encode_categoricals(df):
    cat_cols = ["Area", "VehBrand", "VehGas", "Region"]
    
    df = pd.get_dummies(df, columns=cat_cols, drop_first=True)
    return df

def scale_features(df, features):
    scaler = StandardScaler()
    df[features] = scaler.fit_transform(df[features])
    return df, scaler

train = pd.read_csv("../data/claims_train.csv")
train = feature_selection(train)
train = scale_features(train, features=["VehPower", "VehAge", "DrivAge", "BonusMalus"])[0]
train = encode_categoricals(train)
train = clean_data(train)
train




,ClaimNb,Exposure,VehPower,VehAge,DrivAge,BonusMalus,Area_B,Area_C,Area_D,Area_E,...,Region_R53,Region_R54,Region_R72,Region_R73,Region_R74,Region_R82,Region_R83,Region_R91,Region_R93,Region_R94
0,0,0.43,0.266627,1.935096,-0.672294,2.253198,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
1,0,0.10,0.266627,1.758443,2.440453,2.253198,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
2,0,0.33,0.266627,-0.714696,-0.672294,1.038310,False,False,False,True,...,False,False,False,False,False,True,False,False,False,False
3,0,0.56,-0.708743,-0.538043,1.945243,-0.496285,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,0,0.27,0.754312,-1.244655,-0.601550,-0.624167,False,False,False,True,...,True,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
542405,0,0.20,-0.221058,0.521873,-0.955271,1.038310,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
542406,0,0.06,1.729682,1.228484,-0.813783,0.015247,False,False,True,False,...,False,False,True,False,False,False,False,False,False,False
542407,0,0.34,-0.221058,0.168568,-0.955271,2.253198,False,True,False,False,...,False,False,False,False,False,True,False,False,False,False
542408,0,0.72,1.241997,-0.008085,-0.460062,0.782545,False,False,False,True,...,False,False,False,True,False,False,False,False,False,False
